## **Name :- Karan Nayak** 



## **Project Topic :- Evolving Cellular Automata using GA**






## Kindly do and read the following :- 


*   Run all the cells till Initialize Population
*   Upload the image of the binary hand to test the working of GA. Make sure the bonus is set to "True" in the function and also when you run the function**
*   There are 3 more images given to check the bonus part , kindly upload them and change the name of the image in "cv2.imread" parameter of initialize population . Also set the bonus to "True".
*    Sometimes , depending on the distortion in the start image , the GA overconverges the solution to all 0s. 
*The main algorithm is in **"Main Code"** cell . 
*The GA produces optimal results with 5 generations , after that it starts converging to same solution which is not the desired result. 

**Initially the code is set to directly run the bonus part by using the apple image. SO kindly upload the apple image and change the name of the image in "cv2.imread" parameter of initialize population. Make sure the bonus is set to "True" in the function and also when you run the function**



## Initializations

In [0]:
import json
import numpy as np
import cv2
import math
import random
from random import randint
import copy
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import time

## Binary Image Converter

In [0]:
def binary_image(pop):
  t = 127 #threshold
  start_image = np.array(population[0])
  goal_image = np.array(population[1])
  start_image = start_image.astype(np.uint8) # coercing the image into uint8 to apply cv2 threshold
  goal_image = goal_image.astype(np.uint8) # coercing the image into uint8 to apply cv2 threshold
  sb_image = cv2.threshold(start_image,t,1,cv2.THRESH_BINARY)
  sb_list = sb_image[1].tolist() #converting the array to list incase required in code
  gb_image = cv2.threshold(goal_image,t,1,cv2.THRESH_BINARY)
  gb_list = gb_image[1].tolist() #converting the array to list incase required in code
  return sb_image[1],gb_image[1]

## Rule Applier Function

In [0]:
##Take the current window , the rule table for all the individuals and individual number as reference
##Converts the binary window to integer and then based on that , refers the corresponding rule for 
##the given individual in the rule table

def rule_applier (w_3x3,rules,ind):
  window_9 = w_3x3.reshape(1,9)
  w_str = "".join(str(x) for x in window_9[0])
  w_int = int(w_str,2)
  if rules[ind][w_int] == 1 :
    w_3x3[1][1] = 1
    #print("Rule ", rules[ind][w_int] , " at index " , w_int , " applied on the current window")
    return w_3x3
  else:
    w_3x3[1][1] = 0
    #print("Rule ", rules[ind][w_int] , " at index " , w_int , " applied on the current window")
    return w_3x3

## Individual Fetch

In [0]:
def ind_fetch(pop):
  indi = 2 #starting to take individuals from position '2' in the dictionary
  r1_table = [] #creating a list to store the rules for the individuals stored in r_table in below loop.
  while indi < len(population):
    r_table = [] # list for the rules of an individual. IDEA here is to store the rules for the respective 5 bit binary number. Here 
    ############## a counter c is taken which will be converted to binary and used as reference to fetch it's corresponding rule. 
    c = 0
    for i in range(len(population[indi])): #takes an individual , extracts the rules as per the truth table and store in the list "r_table"
      e = f'{c:09b}'
      var = population[indi][i][e]	
      r_table.append(var)
      c += 1
    
    r1_table.append(r_table)
    indi += 1
  
  return r1_table


## Calculating fitness of binary start and goal images .

In [0]:
def calculate_fitness(final_image, goal_image):
  fitness = 0
  
  for x_pixel in range(len(final_image)):
    for y_pixel in range(len(final_image)):
      if final_image[x_pixel][y_pixel] != goal_image[x_pixel][y_pixel]:
        fitness = fitness + 1

  return fitness

## Visualize Image

In [0]:
def visualize_image(image):
  cv2_imshow(image)

## Window SLider

In [0]:
def window_slider(a1,rules,candidate):

  j = 0
  while j < (len(a1)-2):
    i = 0
    while i < (len(a1[0])-2):
      window = a1[j:j+3 , i:i+3]
      window = rule_applier(window,rules,candidate)
      i += 1
      

    if (i == len(a1[0])-2):
      w1 = a1[j:j+3 , i:len(a1[0])] 
      w2 = a1[j:j+3 , :1]
      window = np.hstack([w1,w2])
      window = rule_applier(window,rules,candidate)
      a1[j:j+3 , i:len(a1[0])] = window[: , :2]
      a1[j:j+3 , :1] = window[: , 2:]
      i += 1
      w1 = a1[j:j+3 , i:len(a1[0])] 
      w2 = a1[j:j+3 , :2]
      window = np.hstack([w1,w2])
      window = rule_applier(window,rules,candidate)
      a1[j:j+3 , i:len(a1[0])] = window[: , :1]
      a1[j:j+3 , :2] = window[: , 1:]

    j += 1


  #covering last row elements

  if j == len(a1)-2:
    i = 0
    while i < (len(a1[0])-2):
      w1 = a1[j:len(a1[0]) , i:i+3]
      w2 = a1[:1 , i:i+3]
      window = np.vstack([w1,w2])
      window = rule_applier(window,rules,candidate)
      a1[j:len(a1[0]) , i:i+3] = window[:2 , :]
      a1[:1 , i:i+3] = window[2: , :]
      i +=1
    if (i == len(a1[0])-2):
      w1 = a1[j:len(a1[0]) , i:len(a1[0])] 
      w2 = a1[j:len(a1[0]) , :1]
      e = np.hstack([w1,w2])
      w3 = a1[:1 , i:len(a1[0])]
      w4 = a1[:1 , :1]
      f = np.hstack([w3,w4])
      window = np.vstack([e,f])
      window = rule_applier(window,rules,candidate)
      a1[j:len(a1[0]) , i:len(a1[0])] = window[:2 , :2]
      a1[j:len(a1[0]) , :1] = window[:2 , 2:]
      a1[:1 , i:len(a1[0])] = window[2: , :2]
      a1[:1 , :1] = window[2: , 2:]
      i += 1
      w1 = a1[j:len(a1[0]) , i:len(a1[0])] 
      w2 = a1[j:len(a1[0]) , :2]
      e = np.hstack([w1,w2])
      w3 = a1[:1 , i:len(a1[0])]
      w4 = a1[:1 , :2]
      f = np.hstack([w3,w4])
      window = np.vstack([e,f])
      window = rule_applier(window,rules,candidate)
      a1[j:len(a1[0]) , i:len(a1[0])] = window[:2 , :1]
      a1[j:len(a1[0]) , :2] = window[:2 , 1:]
      a1[:1 , i:len(a1[0])] = window[2: , :1]
      a1[:1 , :2] = window[2: , 1:]
    j += 1
    

  #covering first row elements 

  if j == len(a1)-1:
    i = 0
    while i < (len(a1[0])-2):
      w1 = a1[j:len(a1[0]) , i:i+3]
      w2 = a1[:2 , i:i+3]
      window = np.vstack([w1,w2])
      window = rule_applier(window,rules,candidate)
      a1[j:len(a1[0]) , i:i+3] = window[:1 , :]
      a1[:2 , i:i+3] = window[1: , :]
      i +=1
    if (i == len(a1[0])-2):
      w1 = a1[j:len(a1[0]) , i:len(a1[0])] 
      w2 = a1[j:len(a1[0]) , :1]
      e = np.hstack([w1,w2])
      w3 = a1[:2 , i:len(a1[0])]
      w4 = a1[:2 , :1]
      f = np.hstack([w3,w4])
      window = np.vstack([e,f])
      window = rule_applier(window,rules,candidate)
      a1[j:len(a1[0]) , i:len(a1[0])] = window[:1 , :2]
      a1[j:len(a1[0]) , :1] = window[:1 , 2:]
      a1[:2 , i:len(a1[0])] = window[1: , :2]
      a1[:2 , :1] = window[1: , 2:]
      i += 1
      w1 = a1[j:len(a1[0]) , i:len(a1[0])] 
      w2 = a1[j:len(a1[0]) , :2]
      e = np.hstack([w1,w2])
      w3 = a1[:2 , i:len(a1[0])]
      w4 = a1[:2 , :2]
      f = np.hstack([w3,w4])
      window = np.vstack([e,f])
      window = rule_applier(window,rules,candidate)
      a1[j:len(a1[0]) , i:len(a1[0])] = window[:1 , :1]
      a1[j:len(a1[0]) , :2] = window[:1 , 1:]
      a1[:2 , i:len(a1[0])] = window[1: , :1]
      a1[:2 , :2] = window[1: , 1:]

    j += 1

  return a1    

## Single point X-over

In [0]:
def xover1(p1,p2):
  decider = randint(1, len(p1)-1)
  print("Xover point was :- " ,decider)
  temp = copy.deepcopy(p1)
  p1 = p1[:decider] + p2[decider:]
  p2 = p2[:decider] + temp[decider:]
  return p1 , p2

## Two point X-over

In [0]:
def xover2(p1,p2):
  l = int((len(p1)-1)/2)
  decider1 = randint(1, l)
  decider2 = randint(l+1, (len(p1)-1))
  print("Xover points are :- " ,decider1 , " and " , decider2)

  temp = copy.deepcopy(p1)
  p1 = p1[:decider1] + p2[decider1:decider2] + p1[decider2:]
  p2 = p2[:decider1] + temp[decider1:decider2] + p2[decider2:]
  return p1 , p2


## Mutation Type 1

In [0]:
def mutation1(fitness,pool,dividend,fitness_min):
  divisor = int(dividend / 512)
  if fitness > (fitness_min-200)  and fitness < (fitness_min+200) :
    n = (int(fitness/divisor))-1
    #print(n)
    sel = random.sample(range(0,len(pool)),n)
    print("Index selected was :- ", sel)
    for i in sel:
      if pool[i] == 0:
        pool[i] = 1
      else:
        continue
    return pool
  
  else:
    print("Performing mutation")
    #for j in range(len(fitness)):
    n = (int(fitness/divisor))-1
    #print(n)
    sel = random.sample(range(0,len(pool)),n)
    print("Index selected was :- ", sel)
    p_m = 0.5 #mutation rate
    v_float = random.uniform(0,1)
    for i in sel:
      if v_float < p_m:
        pool[i] = 0
      else:
        continue
    return pool

## Mutation Type 2

In [0]:
def mutation2(fitness,pool):

  p_m = 0.5 
  v_float = random.uniform(0,1)
  if v_float < 0.5:
    #for j in range(len(fitness)):
    mn = []
    for i in range(9):
      val = 2**i
      mn.append(val)
      if pool[val] == 1:
        pool[val] = 0
      else:
        continue
      
    
    pq = []
    for r in range(4):
      k = randint(2,len(mn)-1)
      temp = random.sample(mn,k)
      val1 = sum(temp)
      pq.append(val1)
      if pool[val1] == 1:
        pool[val1] = 0
      else:
        continue

    ef = mn + pq
    print("Indices of changed elements for ", j ," individual are :- " , ef)    
    return pool
    
  else:
    return pool


## Fitness Evaluator

In [0]:
def fitness_evaluator(pool,start,goal):
  offspring_fitness =[]
  offspring_solutions = [None] * len(pool)
  for i in range(len(pool)):
    trial_start = copy.deepcopy(start)
    for p in range(5):
      trial_start = window_slider(trial_start,pool,i)
    offspring_solutions[i] = trial_start
    o_fit = calculate_fitness(offspring_solutions[i],goal)
    offspring_fitness.append(o_fit)
  
  return offspring_solutions, offspring_fitness

## Initialize Population

In [0]:
################################################################################
#
#   Writes the initial_image and goal_imate to a JSON file. Initializes the 
#   population of candidates with a size of population_size. Writes this 
#   population to the JSON file.
#
#   The JSON file is called "CellularAutomata.json"
#   
#
#   Parameters:
#   initial_image - the greyscale image "binaryhand.png"
#   population_size - an integer defining how many rules tables to generate
#
################################################################################
def initialize_population(initial_image, population_size=3, bonus=True):
    new_population = []
    random_truth_table = []
    if bonus == False:
      a = np.array([[random.uniform(0.25, 1.5), 0.0],
                [0.0, random.uniform(0.25, 1.5)]])
    
      M, N = initial_image.shape
      points = np.mgrid[0:N, 0:M].reshape((2, M*N))
      new_points = np.linalg.inv(a).dot(points).round().astype(int)
      x, y = new_points.reshape((2, M, N), order='F')
      indices = x + N*y

      transformed_image = np.take(initial_image, indices, mode='wrap')
      goal_image = cv2.Canny(transformed_image, 256, 256)

      new_population.append(transformed_image.tolist())
      new_population.append(goal_image.tolist())

    elif bonus == True:
      for x_pixel in range(len(initial_image)):
        for y_pixel in range(len(initial_image[0])):
          if initial_image[x_pixel][y_pixel] < 128:
            initial_image[x_pixel][y_pixel] = 0
          elif initial_image[x_pixel][y_pixel] >= 128:
            initial_image[x_pixel][y_pixel] = 255
      
      goal_image = cv2.Canny(initial_image, len(initial_image), len(initial_image[0]))

      new_population.append(initial_image.tolist())
      new_population.append(goal_image.tolist())

    for individual in range(population_size):
      random_truth_table= list()
      for bit in range(512):
        key = format(bit, 'b')
        while len(key) != 9:
          key = "0" + key
        random_truth_table.append({key: np.random.randint(2)})
      new_population.append(random_truth_table)
    
    with open("CellularAutomata.json", 'w') as o:
      o.write(json.dumps(new_population))

In [0]:
initial_image = cv2.imread("apple.png", 0)

initialize_population(initial_image, population_size =10, bonus = True)

with open("CellularAutomata.json",'r') as f:
  population = json.loads(f.read());

## Main Code

In [0]:
#############################################################################################################
#############################################################################################################
#############################################################################################################
#
#   Parameters that will help understand the code.
#
#   1)a_start = binary of the start image
#   2)a_goal = binary of the goal image
#   3)total_points = total number of points in the image array . Provides flexibility
#   in mutation when any other image is passed . Basically it is to CONSIDER THE BONUS PART
#   4)rules = list of rules . Basically individuals , but just containing the rules ,
#   not the corresponding binary number.
#   5)Best values in each GA :- 
#       - ga_best_fitness
#       - ga_best_solution :- resulting array on applying the individual to the start array .
#       - ga_best_individuals
#   6)Survivor in each GA :- 
#       - ga_fitness
#       - ga_solutions
#       - ga_individuals
#   7)see_start = binary to greyscale of the solution to visualize it. 
#   8)see_goal = binary to greyscale of the solution to visualize it. 
#
#############################################################################################################
#############################################################################################################
#############################################################################################################
start_time = time.clock()

with open("CellularAutomata.json",'r') as f:
  population = json.loads(f.read());

a_start,a_goal = binary_image(population) 
fit_start = calculate_fitness(a_start,a_goal)
print("Fitness before applying rules :- ", fit_start)

total_points = len(a_start) * len(a_start[0])
#total_ones = np.count_nonzero(a_goal == 1)

rules = ind_fetch(population)

ind_count = (len(population)-2)
ga_best_fitness = []
ga_best_solution = []
ga_best_individuals = []
ga_fitness = [] 
ga_solutions = [] 
ga_individuals = []
avg_ga_fitness = []

generations = 5 
for g in range(generations):
  ind_fit = []
  ind_solutions = [None] * ind_count
  for k in range(0,ind_count): #running the loop for all individuals
    passes = 10
    target = copy.deepcopy(a_start)
    for p in range(passes):
      result_array = window_slider(target,rules,k)
    ind_solutions[k] = result_array
    #print("Goal image is :- ", a_goal , sep="\n")
    #print("Final Array :- " , ind_solutions[k], sep="\n")
    ind_fit.append(calculate_fitness(ind_solutions[k],a_goal))

  print("The fitness for all the individuals is :- " , ind_fit)

  fitness_pool = copy.deepcopy(ind_fit)
  ind_pool = copy.deepcopy(rules)


  ##Once the fitness is calculated , do the parent selection  and then apply apply the variation operators . 
  ## i.e. Elite + random  parent selection

  ##Parent selection :- Select two parents with best fitness and one parent randomly. 

  elite_parents = 2
  random_parents = 2
  elite_pool = [None] * elite_parents
  random_pool = [None] * random_parents
  parent_pool = [None] * elite_parents + [None] * random_parents
  parent_fitness = []

  for i in range(0,elite_parents):

    min_fitness = min(fitness_pool)
    print("Fitness was :- " , min_fitness)
    parent_fitness.append(min_fitness)
    fit_index = fitness_pool.index(min_fitness)
    fitness_pool.remove(min_fitness)
    elite_pool[i] = ind_pool.pop(fit_index)


  for j in range(0,random_parents):
    s = random.choices(fitness_pool , k = 1)
    parent_fitness.append(s[0])
    fit_index = fitness_pool.index(s[0])
    fitness_pool.remove(s[0])
    random_pool[j] = ind_pool.pop(fit_index)

  
  parent_pool = elite_pool + random_pool

  #Performing crossover

  offspring_pool = []
  for p in range(len(parent_pool)-2):
    parent = random.sample(parent_pool,2)
    parent1 = parent[0]
    parent2 = parent[1]
    print(parent1)
    print(parent2)
    parent_pool.remove(parent[0])
    parent_pool.remove(parent[1])
    #o1 , o2 = xover1(parent1 , parent2)
    o1 , o2 = xover2(parent1 , parent2)
    offspring_pool.append(o1)
    offspring_pool.append(o2)

  print("Offsprings after xover :- " , offspring_pool)
  offspring_fitness = []

  #Calculating fitness after performing crossover

  offspring_solutions = [None] * len(offspring_pool)
  for i in range(len(offspring_pool)):
    trial_start = copy.deepcopy(a_start)
    for p in range(5):
      trial_start = window_slider(trial_start,offspring_pool,i)
    offspring_solutions[i] = trial_start
    o_fit = calculate_fitness(offspring_solutions[i], a_goal)
    offspring_fitness.append(o_fit)

  print("Fitness after xover :- ", offspring_fitness)

  ## Performing non-uniform mutation with constraints

  mut_min = min(offspring_fitness)
  for i in range(len(offspring_fitness)):
      offspring_pool[i] = mutation1(offspring_fitness[i],offspring_pool[i],total_points,mut_min)
      #offspring_pool[i] = mutation2(offspring_fitness[i],offspring_pool[i],)

  ##Calculating fitness after performing mutation

  offspring_solutions , offspring_fitness = fitness_evaluator(offspring_pool,a_start,a_goal)

  print("Fitness after mutation :- " , offspring_fitness)


  #Survivor selection

  next_gen = len(rules)

  survivor_pool = [None] * next_gen
  survivor_fitness = [None] * next_gen
  survivor_solutions = [None] * next_gen
  fit_select = copy.deepcopy(ind_fit + offspring_fitness)
  sur_select = copy.deepcopy(rules + offspring_pool)
  sol_select = copy.deepcopy(ind_solutions + offspring_solutions)

  offspring_fitness.remove(min(offspring_fitness))
  selector = int(len(survivor_pool)*20 / 100) 
  
  for i in range(len(survivor_pool)-selector):
    fit_min = min(fit_select)  
    var = fit_select.index(fit_min)
    survivor_pool[i] = sur_select.pop(var)
    survivor_fitness[i] = fit_select.pop(var)
    survivor_solutions[i] = sol_select.pop(var)
  
  for j in range(len(survivor_pool)-selector , len(survivor_pool)):
    var1 = randint(0,len(fit_select)-1)
    survivor_pool[j] = sur_select.pop(var1)
    survivor_fitness[j] = fit_select.pop(var1)
    survivor_solutions[j] = sol_select.pop(var1)

  
  rules = survivor_pool

  print("Fitnesses in GA " , g , " are :- " , survivor_fitness )
  print(sep="\n")
  print("**********************END OF GA ", g , "*************************")
  print(sep="\n\n")
  best_fitness = min(survivor_fitness)
  best_indx = survivor_fitness.index(best_fitness)
  best_solution = survivor_solutions[best_indx]
  best_individual = survivor_pool[best_indx]
  ga_best_fitness.append(best_fitness)
  ga_best_solution.append(best_solution)
  ga_best_individuals.append(best_individual)
  ga_fitness.append(survivor_fitness) 
  ga_individuals.append(survivor_pool)
  ga_solutions.append(survivor_solutions)


    
print("GA best fitnesses are :- " , ga_best_fitness)

t = 0.5
min_of_all = min(ga_best_fitness)
print("The solution with best fitness is :- " , min_of_all)
min_indx = ga_best_fitness.index(min_of_all)
print("The solution is :- " , ga_best_solution[min_indx] , sep="\n")
print("The individua is :- " , ga_best_individuals[min_indx] , sep="\n")
see_goal = cv2.threshold(a_goal,t,255,cv2.THRESH_BINARY)
see_final = cv2.threshold(ga_best_solution[min_indx],t,255,cv2.THRESH_BINARY)
print("The goal image is :- " , sep="\n")
visualize_image(see_goal[1])
print("The final image is :- " , sep="\n")
visualize_image(see_final[1])


x1 = np.arange(0,generations , 1)
x2 = np.arange(0,len(ga_fitness[0]) , 1)


plt.plot(x1 , ga_best_fitness , 'bs')
plt.xlabel('Generation')
plt.ylabel('Best Fitness In Each GA')
plt.show()

for i in range(len(ga_fitness)):
  avg = sum(ga_fitness[i]) / len(ga_fitness[i])
  avg_ga_fitness.append(avg)

plt.plot(x1[0], avg_ga_fitness[0], 'r^', x1[1], avg_ga_fitness[1], 'b^', x1[2], avg_ga_fitness[2],\
         'g^' , x1[3], avg_ga_fitness[3], 'rs', x1[4], avg_ga_fitness[4], 'bs')

plt.xlabel('Generations')
plt.ylabel('Avg Fitness')
plt.show()

plt.figure()
plt.subplot(511)
plt.plot(x2, ga_fitness[0], 'rs')
plt.xlabel('Generations')
plt.ylabel('Fitness of individuals')
plt.show()

plt.subplot(512)
plt.plot(x2, ga_fitness[1], 'gs')
plt.xlabel('Generations')
plt.ylabel('Fitness of individuals')
plt.show()

plt.subplot(513)
plt.plot(x2, ga_fitness[2], 'bs')
plt.xlabel('Generations')
plt.ylabel('Fitness of individuals')
plt.show()

plt.subplot(514)
plt.plot(x2, ga_fitness[3], 'r^')
plt.xlabel('Generations')
plt.ylabel('Fitness of individuals')
plt.show()

plt.subplot(515)
plt.plot(x2, ga_fitness[4], 'g^')
plt.xlabel('Generations')
plt.ylabel('Fitness of individuals')
plt.show()



time_taken = time.clock() - start_time
print("Execution time till here was :- " , time_taken)



Fitness before applying rules :-  5077
The fitness for all the individuals is :-  [19374, 18711, 17765, 18199, 17782, 19113, 18541, 19035, 17382, 19136]
Fitness was :-  17382
Fitness was :-  17765
[0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1

KeyboardInterrupt: ignored